In [16]:
import re
import pprint

data = '''
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
'''

splitted = data.strip().splitlines()
splitted[0] = '$ cd root' # natvrdo zrušiť ten character '/' 

In [17]:
splitted

['$ cd root',
 '$ ls',
 'dir a',
 '14848514 b.txt',
 '8504156 c.dat',
 'dir d',
 '$ cd a',
 '$ ls',
 'dir e',
 '29116 f',
 '2557 g',
 '62596 h.lst',
 '$ cd e',
 '$ ls',
 '584 i',
 '$ cd ..',
 '$ cd ..',
 '$ cd d',
 '$ ls',
 '4060174 j',
 '8033020 d.log',
 '5626152 d.ext',
 '7214296 k']

In [18]:
# máme mená aj počet directories
ppath = dict()
nazov = ''
counter = 0
for line in splitted:
    kdesme = splitted.index(line)
    if line.startswith('$ cd') and (line != '$ cd ..'):
        nazov = line
        ppath[counter] = { "nazov": nazov , "file_size" : [ ] , "sum":0 , "file" : [ ] , "dirs" : [] , "cumm":0 } 
        counter += 1

print("counter =",counter)
print(len(ppath))
assert len(ppath) == 4
# assert len(ppath) == 184

counter = 4
4


In [19]:
ppath

{0: {'nazov': '$ cd root',
  'file_size': [],
  'sum': 0,
  'file': [],
  'dirs': [],
  'cumm': 0},
 1: {'nazov': '$ cd a',
  'file_size': [],
  'sum': 0,
  'file': [],
  'dirs': [],
  'cumm': 0},
 2: {'nazov': '$ cd e',
  'file_size': [],
  'sum': 0,
  'file': [],
  'dirs': [],
  'cumm': 0},
 3: {'nazov': '$ cd d',
  'file_size': [],
  'sum': 0,
  'file': [],
  'dirs': [],
  'cumm': 0}}

In [20]:
counter = 0
for line in splitted:
    if line.startswith('dir'):
        counter += 1
        
print(counter)

3


In [21]:
counter = -1
for line in splitted:

    if (line.startswith('$ cd')) and (line != '$ cd ..') : # názov adresára
        counter += 1
        dir_index = counter # splitted.index(line) # line # napr. '$ cd root'
        print(dir_index)
    
    if line == '$ ls':
        pass
    
    if line.startswith('dir') == 'dir':
       nazov_d = line[4:] # máme názov
       ppath[dir_index]['dirs'].append(nazov_d)
    
    if line[0].isdecimal(): # súbor
        najdene = re.match('\d+', line) # file
        najdene_size = int(najdene.group()) # meno resp. velkost suboru
        print(counter)
        ppath[dir_index]['file_size'].append(najdene_size)
        ppath[dir_index]['file'].append(line)
        ppath[dir_index]['sum'] = sum(ppath[dir_index]['file_size'])

0
0
0
1
1
1
1
2
2
3
3
3
3
3


In [22]:
# počiatočný cumm nastavíme na sum
for i in ppath:
    ppath[i]['cumm'] += ppath[i]['sum']

In [23]:
pprint.pprint(ppath)

{0: {'cumm': 23352670,
     'dirs': [],
     'file': ['14848514 b.txt', '8504156 c.dat'],
     'file_size': [14848514, 8504156],
     'nazov': '$ cd root',
     'sum': 23352670},
 1: {'cumm': 94269,
     'dirs': [],
     'file': ['29116 f', '2557 g', '62596 h.lst'],
     'file_size': [29116, 2557, 62596],
     'nazov': '$ cd a',
     'sum': 94269},
 2: {'cumm': 584,
     'dirs': [],
     'file': ['584 i'],
     'file_size': [584],
     'nazov': '$ cd e',
     'sum': 584},
 3: {'cumm': 24933642,
     'dirs': [],
     'file': ['4060174 j', '8033020 d.log', '5626152 d.ext', '7214296 k'],
     'file_size': [4060174, 8033020, 5626152, 7214296],
     'nazov': '$ cd d',
     'sum': 24933642}}


In [24]:
# buffer! jednoduchý
buffer = [ ]
counter = -1
for line in splitted:
        if (line.startswith('$ cd')) and (line != '$ cd ..') : # názov adresára
            counter +=1
            index = counter
            buffer.append(index)
            print(buffer)

        if (line == '$ cd ..'): 
            what = buffer.pop()
            where = buffer[-1]           
            print(buffer)
        
        if (line == splitted[-1]):
            for rollup in range(len(buffer)-1):
                what = buffer.pop()
                where = buffer[-1]
                print(buffer)

print(len(buffer))

[0]
[0, 1]
[0, 1, 2]
[0, 1]
[0]
[0, 3]
[0]
1


In [25]:
# teraz zapracujem presuny size
buffer = [ ]
counter = -1
for line in splitted:
        if (line.startswith('$ cd')) and (line != '$ cd ..') : # názov adresára
            counter += 1
            index = counter
            buffer.append(index)
            print(buffer)

        if (line == '$ cd ..'): 
            print("back!")
            what = buffer.pop()
            where = buffer[-1]
            print("popping what", what )
            print("popping where", where)
            
            ppath[where]['cumm'] += ppath[what]['cumm'] # tu bolo 'sum'

            print(buffer)
        
        if (line == splitted[-1]):
            for rollup in range(len(buffer)-1):
                what = buffer.pop()
                where = buffer[-1]
                ppath[where]['cumm'] += ppath[what]['cumm']
                print(buffer)

print(len(buffer))

[0]
[0, 1]
[0, 1, 2]
back!
popping what 2
popping where 1
[0, 1]
back!
popping what 1
popping where 0
[0]
[0, 3]
[0]
1


In [26]:
pprint.pprint(ppath)

{0: {'cumm': 48381165,
     'dirs': [],
     'file': ['14848514 b.txt', '8504156 c.dat'],
     'file_size': [14848514, 8504156],
     'nazov': '$ cd root',
     'sum': 23352670},
 1: {'cumm': 94853,
     'dirs': [],
     'file': ['29116 f', '2557 g', '62596 h.lst'],
     'file_size': [29116, 2557, 62596],
     'nazov': '$ cd a',
     'sum': 94269},
 2: {'cumm': 584,
     'dirs': [],
     'file': ['584 i'],
     'file_size': [584],
     'nazov': '$ cd e',
     'sum': 584},
 3: {'cumm': 24933642,
     'dirs': [],
     'file': ['4060174 j', '8033020 d.log', '5626152 d.ext', '7214296 k'],
     'file_size': [4060174, 8033020, 5626152, 7214296],
     'nazov': '$ cd d',
     'sum': 24933642}}


In [27]:
# najviac 100000
for i in ppath:
    if ppath[i]['cumm'] <= 100000:
        print(ppath[i])

{'nazov': '$ cd a', 'file_size': [29116, 2557, 62596], 'sum': 94269, 'file': ['29116 f', '2557 g', '62596 h.lst'], 'dirs': [], 'cumm': 94853}
{'nazov': '$ cd e', 'file_size': [584], 'sum': 584, 'file': ['584 i'], 'dirs': [], 'cumm': 584}


In [28]:
# najviac 100000
vysledok = 0
for i in ppath:
    if ppath[i]['cumm'] <= 100000:
        vysledok += ppath[i]['cumm']

print("vysledok1:", vysledok)
assert vysledok == 95437

vysledok1: 95437


# Part2:

In [29]:
# f'{1000000:,}'

disc_size = 70000000
print(f"disc_size: {disc_size:,}")

total_cumm = ppath[0]['cumm']
print(f"occupied: {total_cumm:,}")

unused = disc_size - total_cumm # 70 000 000
print(f"unused: {unused:,}")

space_needed = 30000000
needed = space_needed - unused # 30 000 000
print(f"space_needed: {space_needed:,}")
print(f"needed: {needed:,}")

assert needed == 8381165

disc_size: 70,000,000
occupied: 48,381,165
unused: 21,618,835
space_needed: 30,000,000
needed: 8,381,165


In [30]:
mlist = [ ]
for i in ppath:
    if ppath[i]['cumm'] > needed:
        # print(ppath[i]['cumm'])
        mlist.append(ppath[i]['cumm'])

# print(mlist)
vysledok2 = min(mlist)
assert vysledok2 == 24933642
print("vysledok2 =", vysledok2)

vysledok2 = 24933642
